# Instant Customization

This notebook demonstrates how to use one or more reference images to create a new image, a feature referred to as "instant customization".

The "IMAGE_VARIATION" task type accepts up to five images as references. The output image will incorporate the style, subject, or other characterstics of the reference images based on what is emphasized in your text prompt. In this example, we use the three images below as references and the prompt *"a red haired boy sits in a school classroom looking bored, illustrated story"*.

<div style="display: flex; justify-content: space-between;">
    <div style="width: 31%;">
        <p align="center">
            <img src="../images/redhair-boy-1.png" width="100%" style="padding: 4px">
            <br>
            <em>Reference 1</em>
        </p>
    </div>
    <div style="width: 31%;">
        <p align="center">
            <img src="../images/redhair-boy-2.png" width="100%" style="padding: 4px">
            <br>
            <em>Reference 2</em>
        </p>
    </div>
    <div style="width: 31%;">
        <p align="center">
            <img src="../images/redhair-boy-3.png" width="100%" style="padding: 4px">
            <br>
            <em>Reference 3</em>
        </p>
    </div>
</div>

You will get an output similar to the following:

<img src="../images/doc-images/instant-customization-example-1.png" width="100%" style="padding: 4px">

Try changing the prompt to put this character in different situations or even change to a differernt character while retaining the visual style.

In [ ]:
from random import randint
from amazon_image_gen import BedrockImageGenerator
import file_utils
import logging
import base64
from datetime import datetime

logging.basicConfig(level=logging.INFO, format='[%(levelname)s] %(message)s')

reference_image_paths = [
    "../images/redhair-boy-1.png",
    "../images/redhair-boy-2.png",
    "../images/redhair-boy-3.png",
]

# Load all reference images as base64.
images = []
for path in reference_image_paths:
    with open(path, "rb") as image_file:
        images.append(base64.b64encode(image_file.read()).decode("utf-8"))

# Configure the inference parameters.
inference_params = {
    "taskType": "IMAGE_VARIATION",
    "imageVariationParams": {
        "images": images, # Images to use as reference
        "text": "a red haired boy sits in a school classroom looking bored, illustrated story", 
        "negativeText": "soccer ball, ball",
        "similarityStrength": 0.9,  # Range: 0.2 to 1.0
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,  # Number of variations to generate. 1 to 5.
        "quality": "standard",  # Allowed values are "standard" and "premium"
        "width": 1280,  # See README for supported output resolutions
        "height": 720,  # See README for supported output resolutions
        "cfgScale": 4.0,  # How closely the prompt will be followed
        "seed": randint(0, 858993459),  # Use a random seed
    },
}

# Define an output directory with a unique name.
generation_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output_directory = f"output/{generation_id}"

# Create the generator.
generator = BedrockImageGenerator(
    output_directory=output_directory
)

# Generate the image(s).
response = generator.generate_images(inference_params)

if "images" in response:
    # Save and display each image
    images = file_utils.save_base64_images(response["images"], output_directory, "image")
    for image in images:
        display(image)